<a href="https://colab.research.google.com/github/codyub/ESAA/blob/main/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%94%A5%EB%9F%AC%EB%8B%9D_%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C_ch3_1~3_pg_50~86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 딥러닝 준비


# 1-1 주요 용어
# 1-1-1 하이퍼파라미터
- 딥러닝의 하이퍼 파라미터는 파이썬 프로그래밍과 다름
- 모델이 훈련하면서 가중치와 편향은 자동으로 업데이트되고 조정되지만, 그 외 학습 속도나 반복 훈련 횟수 등 사용자가 직접 설정해야 하는 값들이 매우 많음
- 모델의 예측 성능을 높이기 위해 하이퍼파라미터 값들을 조절하는 데 많은 시간을 투자하는 것이 일반적 -> 튜닝
- 딥러닝은 초기보다 마지막에 튜닝을 하는 것이 좋음

# 1-1-2 과소적합 vs 과대적합
- 머신러닝 학습은 훈련/테스트 데이터로 구분
- 예측할 데이터가 모델이 학습한 데이터와 다른 분포를 갖거나, 학습한 데이터가 특정 레이블에 편향된 데이터로 이뤄져 있는다면 모델이 학습한 후 예측 성능이 현저히 떨어지게 됨
- 모델이 충분히 학습되지 않은 경우 예측 성능이 떨어짐 -> 과소적합
- 반대로 학습 데이터를 지나치게 반복 학습 -> 과대적합 -> 중요한 예측 데이터에 대한 에측 오차 발생

# 1-1-3 에포크
- 딥러닝은 데이터셋을 학습하는 과정을 여러 번 반복하면서 최적의 모델 가중치를 찾는 과정
- 딥러닝 모델이 반복 훈련을 할 때 데이터셋을 전부 학습에 사용하는 1회의 훈련 루프 = 1에포크
- 사전에 설정된 최대 epoch 횟수까지 반복적으로 진행하고 종료
- 과소적합 -> 다시 학습할 필요
- 빠르게 과대적합 구간으로 진입 -> 에포크 줄여 최적 구간에서 학습 조기에 종료시킴

# 1-1-4 손실함수
- 예측 값과 정답 값의 차이 또는 오차 
- 딥러닝 모델에 적용하는 손실 함수는 사용자가 정의
- 지도학습의 분류는 이진분류와 다중분류
- 이진 분류의 경우 활성화 함수가 sigmoid
- 다중 분류의 경우 클래스가 원핫 벡터인 경우 사용, 정답 클래스가 서로다른 숫자로 구분되어 있으면 다른 함수를 사용하고 출력층 활성함수로는 softmax
- 회귀 모델의 경우 MSE, MAE, 후버 등을 사용한다. 


- 다음 예제와 같이 클래스 및 함수 형태로 정의되어 있는 손실 함수를 지정하여 사용

In [ ]:
import tensorflow as tf

# 클래스
tf.keras.losses.BinaryCrossentropy()

# 함수
tf.keras.losses.binary_crossentropy

# 1-1-5 경사하강법(gradient descent)
- 딥러닝 모델을 훈련할 때 모델 내부의 가중치에 대한 미분 값을 구하고, 목적 함수 값이 낮아지는 방향으로 차감하면서 결국 최소 함수 값을 갖도록 하는 방법이다. 
- 딥러닝 훈련 프로세스에서 경사하강법 알고리즘을 적용해 손실함수 값을 최소로 만드는 가중치를 찾을 수 있다. 
- 현재 공개된 다양한 최적화 함수의 근간이 되는 알고리즘
- 경사하강법을 넘파이로 구현해보자


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 샘플에 활용할 데이터셋 만들기
def make_linear(w=0.5,b=0.8, size=50, noise=1.0):
  x = np.random.rand(size)
  y = w * x + b
  noise = np.random.uniform(-abs(noise), abs(noise), size=y.shape)
  yy = y + noise
  plt.figure(figsize=(10,7))
  plt.plot(x,y,color='r',label=f'y = {w}*x + {b}')
  plt.scatter(x,yy,label='data')
  plt.legend(fontsize=20)
  plt.show()
  print(f'w:{w},b:{b}')
  return x,yy

x,y = make_linear(w=0.3, b=0.5, size=100, noise=0.01)

- 그레디언트 w,b를 업데이트 수식을 구했다면 넘파이로 최적화 하는 과정을 구현
- 가중치가 난수 값에 의해 다르게 초기화되기 때문에, 매번 실행할 때마다 수렴하는 속도는 차이가 있다. 

In [ ]:
## 최적화 과정 구현

# 최대 반복 횟수
num_epoch = 1000

# 학습률(learning_rate)
learning_rate = 0.005

# 에러 기록
errors = []

# random 한 값으로 w,b를 초기화
w = np.random.uniform(low=0.0, high=1.0)
b = np.random.uniform(low=0.0, high=1.0)

for epoch in range(num_epoch):
  # Hypothesis 정의
  y_hat = w * x + b

  # Loss Function 정의
  error = 0.5*((y_hat-y)**2).sum()
  if error < 0.005:
    break
  # Gradient 미분 계산
  w = w - learning_rate * ((y_hat-y)*x).sum()
  b = b - learning_rate * (y_hat-y).sum()

  errors.append(error)

  if epoch % 5 == 0:
    print("{0:2} w = {1:.5f}, b = {2:.5f} error = {3:.5f}".format(epoch,w,b,error))

print("----" * 15)
print("{0:2} w = {1:.1f}, b = {2:.1f} error = {3:.5f}".format(epoch,w,b,error))

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(errors)
plt.xlabel('Epochs')
plt.ylabel('Error')
plt.show()

# 1-2 딥러닝 프로세스

딥러닝 모델 구축 및 훈련을 위해서는 다음과 같은 프로세스를 따른다. 
데이터 로드 -> 데이터 전처리 -> 데이터 분할 -> 모델생성&컴파일 -> 훈련 -> 검증 -> 예측(이 사이에 하이퍼파라미터 튜닝도 함)

# 1-2-1 데이터 전처리
- 데이터셋의 종류와 적용 문제에 따라 전처리 방법이 다양함
- 배열의 차원을 변경하거나 스케일 조정 가능
- 데이터셋의 형태나 차원을 미리 구상해야 다음 단계에서 모델을 설계할 때 입력할 데이터셋의 형태를 올바르게 정의 가능

# 1-2-2 모델 생성
- 모델의 구조를 정의학 생성하는 단계
- 순차적 구조 모델, 다중 입력 및 출력을 갖고 복잡한 구조를 갖는 모델 등

# 1-2-3 모델 컴파일
- 딥러닝 모델의 구조를 정의하고 생성한 뒤, 생성된 모델 훈련에 사용할 손실함수, 옵티마이저, 평가지표 등을 정의

# 1-2-4 모델 훈련
- 모델을 훈련하는 단계
- fit()
- 모델 훈련에 필요한 정보를 매개변수로 전달: 훈련 데이터셋, 검증 데이터셋, 에포크, 배치, 크기 및 콜백 함수 등을 지정

# 1-2-5 모델 검증
- 훈련이 완료된 모델을 검증
- 모델을 훈련할 때 사용하지 않은 검증 데이터셋을 모델에 입력하고 모델의 예측 값을 정답과 비교하여 평가지표를 계산
- 반환된 검증 결과를 토대로 잠재적인 모델의 성능 평가가 이뤄짐

# 1-2-6 모델 예측

# 02 단순 신경망 훈련

# 2-1 선형 회귀
- 하나 이상의 독립변수들이 종속변수에 미치는 영향을 추정하는 통계 기법
- 그 중에서도 단순선형회귀 모형은 하나의 X가 Y에 미치는 영향을 추정하며 1차 함수 관계로 나타낼 수 있음
- 가중치 w를 기울기 a로 하고, 편향 b(bias)를 이용해 구한다. 


# 2-2 뉴런
- 인공신경망 모델을 구성하는 하나의 신경을 의미
- 신경망은 여러개의 레이어를 가지며, 1개의 레이어는 1개 이상의 뉴런으로 구성
- 뉴런의 내부에는 가중치가 존재하고, 모델이 훈련을 진행하면서 오차 역전파를 통해 뉴런의 가중치가 업데이트 됨


# 2-3 Dense 레이어
- Dense 레이어는 심층 신경망 모델을 구성하는 가장 기본 레이어
- 각 레이어와 레이어 사이에 모든 뉴런이 서로 연결되어 있으므로, 완전 연결층이라고 부름
- 입력 레이어/출력 레이어는 1개이고, 은닉 레이어는 여러개가 될 수 있다. 각 레이어는 1개 이상의 뉴런을 갖음
- 모든 뉴런들이 정보의 흐름을 나타내는 화살표로 연결되어 있어서 완전 연결층이라고 정의함
- Dense 레이어를 구성하는 노드 개수와 활성화 함수 지정 가능


In [ ]:
import tensorflow as tf

# 10개의 노드로 이루어진 Dense 레이어
tf.keras.layers.Dense(10)

# ReLU 활성화 함수 적용
tf.keras.layers.Dense(10, activation='relu') # 노드개수, 활성화함수
     

# 2-4 데이터셋 만들기
- 단순선형회귀 모형을 지도학습 방식으로 훈련하기 위해 x,y 데이터를 생성
- 넘파이 arrange() 메서드를 활용해 5개의 순차적인 x 데이터를 생성하고, 임의의 1차 함수인 y= 3x+2 관계식에 대입해 y 데이터 5개 생성

In [ ]:
# 샘플 데이터셋 생성
x = np.arange(1,6)

# y = 3x + 2
y = 3*x + 2
print(x)
print(y)

- 생성한 데이터를 시각화

In [ ]:
import matplotlib.pyplot as plt

# 시각화
plt.plot(x,y)
plt.title('y = 3x + 2')
plt.show()

# 2-5 Sequential API
- 텐서플로 케라스가 생성하는 모델 3개 중 하나
- 층을 이어 붙이듯 시퀀스에 맞게 일렬로 연결하는 방식
- 입력 레이어부터 출력 레이어까지 순서를 갖음
- 입력 데이터는 시퀀스의 가장 앞에 위치한 층에 투입, 순서대로 각 층을 하나씩 통과하면서 딥러닝 연산 수행
- 가장 간단하고 직관적인 방법
- 2개 이상의 다중 입력이나 다중 출력을 갖는 복잡한 구조를 만들 수 없다는 단점

# 2-5-1 모델 구조
1. 리스형 정의로 클래스 함수에 파이썬 리스트 형태로 여러 개의 층 입력
2. 앞에 위치한 층부터 연산을 먼저 처리하고 다음 층의 연산 수행
- 아래는 3개의 층을 갖는모델

In [ ]:
## 모델 생성 (Dense 레이어 3개를 갖는 모델)
# 방법1: 리스트형
import tensorflow as tf
model = tf.keras.Sequential([
        tf.keras.layers.Dense(10),
        tf.keras.layers.Dense(5),
        tf.keras.layers.Dense(1)
])

- add 함수로 레이어 추가로 층 추가

In [ ]:
# 방법2: add 함수로 레이어 추가
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(5))
model.add(tf.keras.layers.Dense(1))

# 2-5-2 입력 데이터 형태
- 모델을 구성할 때 반드시 첫 번째 층은 input_shape 매개 변수를 지정
- input_shape 매개변수는 주입할 데이터셋의 shape을 튜플 혹은 리스트로 지정할 수 있음

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
  # 입력 데이터의 shape=(150,4)인 경우
  tf.keras.layers.Dense(10, input_shape=[4]),
  tf.keras.layers.Dense(5),
  tf.keras.layers.Dense(1)
])

# 2-5-3 단순선형회귀 모델 정의
- 1개의 뉴런을 가지는 Dense 레이어 한 개를 사용
- 여기에 정의한 1개의 뉴런은 기울기에 해당하는 w와 상수항에 해당하는 b를 매개변수로 갖음

In [ ]:
# 단순선형회귀 모델 생성
model = tf.keras.Sequential([
        tf.keras.layers.Dense(1, input_shape=[1])
])

# 2-6 모델 요약
- 모델의 구조 확인, 층별 노드 개수, 요약하단에는 훈련 시 업데이트할 파라미터의 개수 표기

In [ ]:
model.summary()

# 2-7 컴파일
- 모델의 훈련 과정에서 적용할 옵티마이저, 손실함수, 평가지표 등을 정의
- 클래스 인스턴스로 지정할 경우 학습률, 모멘텀 등 하이퍼파라미터를 사용자가 직접 지정 가능
- 하지만, 문자열로 지정하는 경우 기본값으로 설정된 하이퍼파라미터를 사용

In [ ]:
# 방법1: 긴 문자열 지정
model.compile(optimizer='sgd', loss='mean_squared_error',
              metrics=['mean_squared_error', 'mean_absolute_error'])

In [ ]:
# 방법2: 짧은 문자열 지정
model.compile(optimizer='sgd', loss='mse', metrics=['mse', 'mae'])

In [ ]:
# 방법3: 클래스 인스턴스 지정
model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.005),
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError(),
                       tf.keras.metrics.MeanSquaredError()
              ])

- 앞서 정의한 단순선형회귀 모델을 훈련시키기 위해 옵티마이저는 확률적 경사하강법 알고리즘인 sgd, 손실함수는 mse, 평가지표는 mae를 지정

In [ ]:
# 컴파일
model.compile(optimizer='sgd', loss='mse', metrics=['mae'])

# 2-8 훈련
- 생성한 모델 인스턴스에 fit() 메서드를 적용하면, 데이터를 입력해 모델을 훈련할 수 있음
- 모델을 훈련한다는 의미는 가중치 텐서를 업데이트 하는 과정
- fit()메서드에는 훈련용 데이터 셋의 입력과 정답에는 출력, 그리고 반복 훈련할 에포크 수를 지정
- 매 에포크가 끝날 때마다 훈련 데이터셋에 대한 손실과 평가 지표를 출력


In [ ]:
# epoch=5: 5번 반복 훈련한 뒤 결과 확인
model.fit(x,y,epochs=5)

In [ ]:
## 코드 종합

# 단순 선형회귀 모델 생성
model = tf.keras.Sequential([
        tf.keras.layers.Dense(1, input_shape=[1])
])

# 컴파일
model.compile(optimizer='sgd', loss='mse', metrics=['mae'])

# 훈련
history = model.fit(x, y, epochs=1200)

In [ ]:
## epoch별 훈련 손실 및 평가지표 시각화
import matplotlib.pyplot as plt

# 20 epoch까지 Loss 수렴에 대한 시각화
plt.plot(history.history['loss'],label='loss')
plt.plot(history.history['mae'],label='mae')
plt.xlim(-1,20)
plt.title('Loss')
plt.legend()
plt.show()

# 2-9 검증
- 모델 인스턴스의 evaluate()메서드로 모델의 성능 검증 가능
- 일반적으로 검증 데이터셋을 입력해 검증
- 별도의 검증셋을 만들지 않았으므로 훈련 데이터셋의 x,y를 대입해 반환되는 결과 확인

In [ ]:
model.evaluate(x,y)

# 2-10 예측
- 훈련이 완료된 모델 인스턴스의 predict() 메서드에 새로운 입력 데이터를 넣어 주면 모델의 예측값을 얻을 수 있음 

In [ ]:
model.predict([10])

# 03 | 심층 신경망으로 이미지 분류
- 이미지 분류를 위해서는 단층이 아닌 더 깊은 층을 쌓아 깊이가 있는 딥러닝 모델을 생성함
- 훈련에 필요한 데이터셋은 케라스 내장 데이터셋을 가져와서 사용
- 분류에 적합한 손실함수를 적용하고 결과 확인


# 3-1 케라스 내장 데이터셋 로드
- 케라스는 모델 훈련에 필요한 샘플 데이터셋을 제공
- 데이터셋은 tensorflow, keras, datasets 서브패키지에 위치하며 지원하는 목록은 다음과 같음

1. boston_housing : 보스톤 주택 가격 데이터셋(회귀)
2. cifar10 : CIFAR10 이미지 분류 데이터셋(분류)
3. cifar100 : CIFAR100 이미지 분류 데이터셋(분류)
4. mnist : mnist 손글씨 데이터셋(분류)
5. fashion_mnist : 의류 데이터셋(분류)
6. imdb : IMDb 영화 데이터셋(분류)
7. reuters : Reuters 뉴스 토픽(분류)

- 케라스의 datasets 패키지에 있는 데이터를 불러오기 위해 load_data() 메소드를 사용

In [ ]:
import tensorflow as tf

# 케라스의 내장 데이터셋에서 mnist 데이터셋 로드
mnist = tf.keras.datasets.mnist

In [ ]:
# load_data()로 데이터셋 로드
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [ ]:
# 로드된 데이터셋 확인
print('train set:', x_train.shape, y_train.shape)
print('test set:', x_test.shape, y_test.shape)

In [ ]:
## 데이터 시각화
import matplotlib.pyplot as plt

# canvas 생성
fig, axes = plt.subplots(3,5)
fig.set_size_inches(8,5)

for i in range(15):
  ax = axes[i//5, i%5]
  # imshow로 이미지 시각화
  ax.imshow(x_train[i], cmap='gray')
  ax.axis('off')
  ax.set_title(str(y_train[i]))

plt.tight_layout()
plt.show()

# 3-2 데이터 전처리
- 이미지 데이터에 대하여 정규화를 통해 데이터의 범위를 조절
- 배열의 각 원소는 이미지를 구성하는 하나의 픽셀 값과 일대일 매칭

In [ ]:
# x_train 배열의 데이터 확인
x_train[0, 10:15, 10:15]

- 데이터의 범위를 0~1 사이의 값을 가지도록 정규화하기 위해 데이터의 최대값인 255로 나눔

In [ ]:
# 픽셀 값의 최소/최대값 확인
print(f'정규화 전] 최소값: {x_train.min()}, 최대값: {x_train.max()}')

# 데이터 정규화
x_train = x_train / x_train.max()

# 정규화 후 최소/최대값 확인
print(f'정규화 후] 최소값: {x_train.min()}, 최대값: {x_train.max()}')

In [ ]:
# test 셋에도 정규화 동일 적용
x_test = x_test / x_test.max()

In [ ]:
# 변환 후 x_train 배열의 데이터 확인
x_train[0, 10:15, 10:15]

# 3-3 Flatten 레이어
- 정규화가 끝난 x_train의 shape은 (60000,28,28)으로, 정규화를 하더라도 개별 데이터 값의 범위는 축소되지만, 원본 배열의 형태는 그대로 유지
- 샘플 이미지 형태는 28,28 2차원이므로, Dense에 넣을 수 없어 1차원으로 전처리해 주입해야 함

In [ ]:
print(f'변경 전 shape:{x_train.shape}')
print(f'ID으로 shape 변경 후: {x_train.reshape(60000,-1).shape}')

- 혹은 다음과 같은 flatten 레이어를 사용해 다차원 데이터를 1차원으로 펼쳐주는 방법도 가능

In [ ]:
print(f'변경 전 shape:{x_train.shape}')

# Flatten 레이어 적용 후 shape 확인
print(f'Flatten 적용 후:{tf.keras.layers.Flatten()(x_train).shape}')

# 3-4 활성화 함수
- 활성화 함수는 입력을 비선형 출력으로 변환해줌
- 일반적으로 선형관계를 나타내는 함수에 비선형을 추가하는 방법으로 표현되기도 함
- 시그모이드, 하이퍼볼릭 탄젠트, ReLu 등

In [ ]:
# Dense 레이어에 relu 활성화 함수를 적용한 경우
tf.keras.layers.Dense(128, activation='relu')

In [ ]:
## 다른 방법
# Dense 레이어와 relu 활성호 함수 레이어를 별도 적용한 경우
model = tf.keras.Sequential([
        tf.keras.layers.Dense(128),
        tf.keras.layers.Activation('relu')
])

# 3-5 입력과 출력
- 딥러닝 모델을 만들 때는 입력 형태를 나타내는 input_shape 매개변수를 지정하는 것이 일반적
- 분류 모델의 가장 마지막 레이어는 출력층으로, 출력층의 노드 개수는 반드시 분류해야 할 클래스 개수와 동일

In [ ]:
model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28,28)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        # 노드 = 10개(클래스 개수와 동일)
        tf.keras.layers.Dense(10, activation='softmax'),
])

- 출력층 노드 개수가 2개 이상인 경우 softmax 활성화 함수를 적용
- 다중분류 문제에서는 softmax 활성화 함수를 사용해야 한다는 의미

In [ ]:
# 출력층 노드 = 1인 경우, sigmoid
tf.keras.layers.Dense(1, activation='sigmoid')

# 출력층 노드 = 2개 이상인 경우, softmax
tf.keras.layers.Dense(10, activation='softmax')

# 3-6 손실함수
- 분류 모델의 손실함수는 모델의 출력층에 따라 올바른 손실함수를 설정해야만 모델이 정상적으로 훈련할 수 있음

In [ ]:
# 이진 분류(출력 노드 개수 = 1, sigmoid인 경우)
model.compile(loss='binary_crossentropy')

In [ ]:
# y가 원핫 벡터인 경우
# [0.,0.,0.,0.,0.,1.,0.,0.,0.,0.]
model.compile(loss='categorical_crossentropy')
# y가 원핫 벡터가 아닌 경우
model.compile(loss='sparse_categorical_crossentropy')

# 3-7 옵티마이저
- 손실을 낮추기 위해 신경망의 가중치와 학습률과 같은 신경망의 속성을 변경하는 데 사용되는 최적화 방법
- 일반적으로 많이 사용되는 알고리즘은 Adam, 대체적으로 좋은 성능을 발휘

In [ ]:
# 방법1: 클래스 인스턴스로 지정
adam = tf.keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=adam)

# 방법2: 문자열로 지정
model.compile(optimizer='adam')

# 3-8 평가지표
accuracy, auc, precision, recall 등



In [ ]:
# 방법1: 클래스 인스턴스로 지정
acc = tf.keras.metrics.SparseCategoricalAccuracy()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=[acc])

In [ ]:
# 방법2: 문자열로 지정
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 3-9 훈련
- 모델을 훈련할 때 검증 셋을 추가 지정하면, 매 epoch마다 훈련 손실과 검증 손실, 그리고 각 셋에 대한 평가지표를 나란히 출력



In [ ]:
model.fit(x_train, y_train,
          # 검증 셋 지정
          validation_data=(x_test, y_test),
          epochs=10)

# 3-10 평가
- 훈련이 종료된 뒤 evaluate() 메서드로 모델 성능을 검증하고 평가 결과를 확인 가능
- 이때 검증 셋인 test를 함수에 전달

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)

print('검증 셋 정확도:', test_acc)

# 3-11 예측
- 훈련된 모델로 새로운 이미지에 대한 분류 값을 예측하는 작업을 수행 가능
- predict()

In [ ]:
predictions = model.predict(x_test)

In [ ]:
# 예측 결과 출력
predictions[0]

In [ ]:
import numpy as np

# 0번 index에 대한 예측 클래스 출력
print(np.argmax(predictions[0]))

# 첫 10개 index에 대한 예측 클래스 출력
print(np.argmax(predictions[:10], axis=1))

In [ ]:
## 데이터 시각화
import matplotlib.pyplot as plt
def get_one_result(idx):
  img, y_true, y_pred, confidence = x_test[idx], y_test[idx], np.argmax(predictions[idx]), 100*np.max(predictions[idx])
  return img, y_true, y_pred, confidence

# canvas 생성
fig, axes = plt.subplots(3,5)
fig.set_size_inches(12,10)
for i in range(15):
  ax = axes[i//5, i%5]
  img, y_true, y_pred, confidence = get_one_result(i)
  # imshow로 이미지 시각화
  ax.imshow(img, cmap='gray')
  ax.set_xticks([])
  ax.set_yticks([])
  ax.set_title(f'True: {y_true}')
  ax.set_xlabel(f'Prediction: {y_pred}\nConfidence: ({confidence:.2f} %)')
plt.tight_layout()
plt.show()